# linear model as baseline

In [3]:
%matplotlib inline
import lab.setup
import functools
import pandas as pd
import numpy as np
import numba

from pandas.tseries.offsets import *

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

g_region_temporal = 5
g_region_spatial  = 1
g_start_date = '2016-03-03'
g_end_date   = '2016-03-03'
g_start_time = '{} 00:00:00'.format(g_start_date)
g_end_time   = '{} 23:59:59'.format(g_end_date)

DATA_PATH = 'dataset'

## extract traffic time in (g_region_temporal, g_region_spatial) as feature vector

### load dataset

In [3]:
topology = pd.read_csv(f'{DATA_PATH}/gy_contest_link_top.txt', sep=';')
ds_train_raw = pd.read_csv(
    f'{DATA_PATH}/gy_contest_link_traveltime_training_data.txt', 
    sep=';', 
    parse_dates=['date'], 
    index_col='date')

In [4]:
print('before sampling: ', ds_train_raw.shape)
ds_train_sample = ds_train_raw.loc[g_start_date:g_end_date]
print('after sampling: ', ds_train_sample.shape)

before sampling:  (7662384, 3)
after sampling:  (86124, 3)


In [ ]:
ds_train = ds_train_sample
# ds_train_sample.groupby('link_ID').count()

### build spatial&& temporal features

In [ ]:
# join topology
ds_train_topo = ds_train.reset_index().merge(topology, on=['link_ID'], how='left')

# parse time interval&& sort dataset
def _parse_time_interval(string):
    return pd.to_datetime(string[1:20])

ds_train_topo['time_intv'] = ds_train_topo.time_interval.apply(_parse_time_interval)
ds_train_topo = ds_train_topo.sort_values(by=['link_ID', 'time_intv']).reset_index(drop=True)

In [ ]:
# fillna
# 先生成一个完整的link_ID+time_intv的grid，然后把数据集join上来，最后fillna
s_link_ID = ds_train_topo.link_ID.unique()
s_time_intv = pd.date_range(
    pd.to_datetime(g_start_time), pd.to_datetime(g_end_time), freq='2min').to_series().reset_index(drop=True)

ds_left  = pd.DataFrame()
ds_right = pd.DataFrame()
ds_left['link_ID'] = s_link_ID
ds_left['cartesian_key'] = 0
ds_right['time_intv'] = s_time_intv
ds_right['cartesian_key'] = 0

ds_full = pd.merge(ds_left, ds_right, on='cartesian_key').drop('cartesian_key', axis=1)
ds_full = pd.merge(ds_full, ds_train_topo, on=['link_ID', 'time_intv'], how='left')
# 1, date, time_interval字段无用，不需要补齐
# 2, in_links, out_links对每个link_ID必然相同，无脑补齐
# 3, travel_time优先使用小的时间戳去补齐大的时间戳，即ffill，对开头的缺失，再使用bfill
fill_col_mask = ['link_ID', 'travel_time', 'in_links', 'out_links']
ds_full[fill_col_mask] = ds_full[fill_col_mask].groupby(by='link_ID').ffill().bfill()
fill_col_mask = ['date', 'time_interval']
ds_full[fill_col_mask] = ds_full[fill_col_mask].fillna('N/A')
ds_full['filled'] = ds_full.date == 'N/A' # 表示是补齐出来的数据
ds_full.head(1)

In [ ]:
# join 1-order uplinks&& downlinks
def _append_col_prefix(df, prefix):
    df.columns = [prefix + '_' + str(i) for i in df.columns]
    
df_uplinks_arr   = ds_full.in_links.str.split('#', expand=True).fillna(0).astype('uint64')
df_downlinks_arr = ds_full.out_links.str.split('#', expand=True).fillna(0).astype('uint64')
_append_col_prefix(df_uplinks_arr, 'uplink')
_append_col_prefix(df_downlinks_arr, 'downlink')

ds_full = pd.concat([ds_full, df_uplinks_arr], axis=1)
ds_full = pd.concat([ds_full, df_downlinks_arr], axis=1)

# join travel time of uplinks&& downlinks(may be null in absence of nbr link)
for nbr_type in ['uplink', 'downlink']:
    for nbr_idx in range(0, 4): # @TODO: magic
        ds_full = pd.merge(
            ds_full, ds_full[['link_ID', 'time_intv', 'travel_time']], 
            left_on=['{}_{}'.format(nbr_type, nbr_idx), 'time_intv'], 
            right_on=['link_ID', 'time_intv'], how='left', 
            suffixes=['', '_{}_{}'.format(nbr_type, nbr_idx)])

# get average travel time of nbrs for each order
nbr_tt_columns = []
for nbr_idx in range(0, 4):
    nbr_tt_columns.append('travel_time_{}_{}'.format('{}', nbr_idx))
    
for nbr_type in ['uplink', 'downlink']:
    ds_full['{}_mean_tt'.format(nbr_type)] \
        = ds_full[[col.format(nbr_type) for col in nbr_tt_columns]].mean(axis=1)

# drop temp columns
temp_cols = []
for nbr_type in ['uplink', 'downlink']:
    for nbr_idx in range(0, 4):
        temp_cols.append('link_ID_{}_{}'.format(nbr_type, nbr_idx))
        temp_cols.append('travel_time_{}_{}'.format(nbr_type, nbr_idx))

ds_full = ds_full.drop(temp_cols, axis=1)
ds_full.isnull().sum()

In [ ]:
ds_full.to_csv('dataset/ds_filled_s1.csv', index=False)

## modeling

In [4]:
# load prepared data
# ds_train_full = pd.read_csv('dataset/ds_s1t5_flatten.csv', dtype={'link_ID':'uint64'}, low_memory=False)
ds_train_full = pd.read_csv('dataset/ds_filled_s1.csv', dtype={'link_ID':'uint64'}, low_memory=False)
ds_train_full.head(1)

,link_ID,time_intv,date,time_interval,travel_time,in_links,out_links,filled,uplink_0,uplink_1,uplink_2,uplink_3,downlink_0,downlink_1,downlink_2,downlink_3,uplink_mean_tt,downlink_mean_tt
0,3377906280028510514,2016-03-03 00:00:00,NaN,NaN,5.1,4377906282541600514,4377906280763800514,True,4377906282541600514,0,0,0,4377906280763800514,0,0,0,55.4,8.4


In [5]:
class PandasDataset(data.Dataset):
    def __init__(self, df, feature_columns):
        self.df = df[feature_columns].astype('float32')
        self.dataset = self.df.values
        self.temporal_order = g_region_temporal
        self.feature_size = self.temporal_order * len(feature_columns)
        self.len = self.df.shape[0] - (self.temporal_order - 1) - 1
        self.label_index = self.df.columns.tolist().index('travel_time')
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        feature_vec = self.dataset[idx:idx+self.temporal_order].reshape(self.feature_size)
        label_vec   = self.dataset[idx+self.temporal_order][self.label_index:self.label_index+1]
        
        return {'feature': feature_vec, 'label': label_vec}

def collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    feature_batch = torch.stack([torch.from_numpy(f['feature']) for f in batch], 0)
    label_batch = torch.stack([torch.from_numpy(f['label']) for f in batch], 0)
    return {
        'feature': feature_batch, 
        'label': label_batch
    }


In [7]:
# Build linear model
feature_columns = ['travel_time', 'uplink_mean_tt', 'downlink_mean_tt']

B     = 256
D_in  = g_region_temporal * len(feature_columns)
D_out = 1
TRAIN_SET_RATIO = 0.8

# Single linear layer
model = torch.nn.Linear(D_in, D_out)
model.cuda()

def rmse(y_hat, y):
    """Compute root mean squared error"""
    return torch.sqrt(torch.mean((y - y_hat).pow(2)))

def mape(y_hat, y):
    """Compute root mean squared error"""
    return torch.mean(((y - y_hat) / y).abs())

loss_fn = torch.nn.L1Loss()
loss_fn = mape

learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 我们需要按link_ID切分dataset，因为不同link的数据不能看作一个时间序列
datasets_train = []
datasets_valid = []
link_no = ds_train_full.link_ID.unique().shape[0]
counter = 0
for link_ID, link_ds in ds_train_full.groupby('link_ID'):
    counter += 1
    if counter < link_no * TRAIN_SET_RATIO:
        datasets_train.append(PandasDataset(link_ds, feature_columns))
    else:
        datasets_valid.append(PandasDataset(link_ds, feature_columns))

print('train set size:', len(datasets_train) * len(datasets_train[0]))
print('valid set size:', len(datasets_valid) * len(datasets_valid[0]))
dataset_train = data.ConcatDataset(datasets_train)
dataset_valid = data.ConcatDataset(datasets_valid)
loader_train = data.DataLoader(dataset_train, batch_size=B, shuffle=False, num_workers=4, collate_fn=collate)
loader_valid = data.DataLoader(dataset_valid, batch_size=B, shuffle=False, num_workers=4, collate_fn=collate)

def validate():
    eval_running_loss = 0.0
    counter = 0
    for i_batch, sample_batch in enumerate(loader_valid):
        x_batch = Variable(sample_batch['feature']).cuda()
        y_batch = Variable(sample_batch['label']).cuda()
        y_batch_pred = model(x_batch)
        loss = loss_fn(y_batch_pred, y_batch)

        eval_running_loss += loss.data[0]
        counter += 1
        
    return eval_running_loss / counter

num_epochs = 100
epoch_loss_records = []
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)
for epoch in range(num_epochs):
    lr_scheduler.step()
    running_loss = 0.0
    counter = 0
    for i_batch, sample_batch in enumerate(loader_train):
        x_batch = Variable(sample_batch['feature']).cuda()
        y_batch = Variable(sample_batch['label']).cuda()

        # forward
        y_batch_pred = model(x_batch)
        loss = loss_fn(y_batch_pred, y_batch)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        counter += 1

    epoch_mean_loss = running_loss / counter
    eval_mean_loss = validate()
    print('=== epoch[{}/{}], loss: {:.6f}, valid_loss: {:.6f} ==='
                  .format(epoch + 1, num_epochs, epoch_mean_loss, eval_mean_loss))
    epoch_loss_records.append(epoch_mean_loss)

train set size: 75075
valid set size: 19305
=== epoch[1/100], loss: 0.517381, valid_loss: 0.270519 ===
=== epoch[2/100], loss: 0.383149, valid_loss: 0.262476 ===
=== epoch[3/100], loss: 0.305735, valid_loss: 0.234680 ===
=== epoch[4/100], loss: 0.322688, valid_loss: 0.411402 ===
=== epoch[5/100], loss: 0.395721, valid_loss: 0.390955 ===
=== epoch[6/100], loss: 0.369979, valid_loss: 0.440615 ===
=== epoch[7/100], loss: 0.381223, valid_loss: 0.437316 ===
=== epoch[8/100], loss: 0.369993, valid_loss: 0.435582 ===
=== epoch[9/100], loss: 0.368392, valid_loss: 0.426705 ===
=== epoch[10/100], loss: 0.368573, valid_loss: 0.426254 ===
=== epoch[11/100], loss: 0.256012, valid_loss: 0.184671 ===
=== epoch[12/100], loss: 0.203575, valid_loss: 0.184315 ===
=== epoch[13/100], loss: 0.202377, valid_loss: 0.182915 ===
=== epoch[14/100], loss: 0.203439, valid_loss: 0.182633 ===
=== epoch[15/100], loss: 0.206585, valid_loss: 0.188532 ===
=== epoch[16/100], loss: 0.206959, valid_loss: 0.182748 ===
=== e